In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#plt.rcParams['figure.figsize'] = [15, 60]

taxon_name = "occs_all"
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
taxa = pd.read_csv(taxon_name+".csv", delimiter=",", 
    usecols=["ORDER", "FAMILY", "GENUS", "SPECIES", "MAX_AGE", "MIN_AGE", "LIDNUM","NAME", "BODYMASS", "DIET_1", "DIET_2", "DIET_3"], 
                   comment='#', na_values="\\N")
#taxa
#print(taxa)
print(taxa.isna().any())
taxa = taxa.fillna(-999)
print(taxa.isna().any())
print(taxa.shape)

indet_sp = taxa['SPECIES']=="indet." 
NA_sp = taxa['SPECIES']==-999 
NA_g = taxa['GENUS']==-999
NA_f = taxa['FAMILY']==-999
not_sp = taxa['SPECIES']=="sp."
indet_genus = taxa['GENUS']=="indet." 
#indet_genus = taxa['GENUS']=="Indet." 
not_genus = taxa['GENUS']=="gen." 
#not_genus = taxa['GENUS']=="Gen." 
inc_sedis_g = taxa['GENUS']=="incertae sedis"
inc_sedis_f = taxa['FAMILY']=="incertae sedis"
bad_min_age = taxa['MIN_AGE'] == -999
bad_max_age = taxa['MAX_AGE'] == -999
nomen_nudum_f = taxa['FAMILY'] == "nomen nudum"
nomen_nudum_g = taxa['GENUS'] == "nomen nudum"
Viverr = taxa['FAMILY']=="Viverravidae" 
bad_age = taxa['MAX_AGE'] > 70
not_ok = NA_g|NA_f| indet_genus | not_genus | inc_sedis_f| inc_sedis_g | bad_min_age | bad_max_age | bad_age| nomen_nudum_f | nomen_nudum_g
#Species data: indet_sp | not_sp | NA_sp|

taxa =taxa[~not_ok]
print("After cleanup:")
print(taxa.shape)

max_ma = np.array(taxa.MAX_AGE)
min_ma = np.array(taxa.MIN_AGE).astype(float)

age_list = np.mean([max_ma, min_ma], axis=0)

name = taxa.GENUS+" "+taxa.SPECIES
name_list = list(name)
#age_list = np.asarray(taxa['mean_ma'])

strat_stages = pd.read_csv("strat_stages.csv", delimiter=",", usecols=["stage", "bottom", "mid", "top", "stg"])
ind = np.digitize(age_list, strat_stages.top, right=True)
stages = strat_stages.stg[ind]
stage_names = strat_stages.stage[ind]

my_stages_bottom = np.round(np.linspace(66, 3, 22), 2) #Roughly 3Myr bins
my_stages_mid = my_stages_bottom-1.5
my_stages_top = my_stages_bottom-3
my_ind = np.digitize(age_list, my_stages_top, right=True)
stage_list = np.linspace(1, 22, 22)
my_stage_numbers = (stage_list[my_ind])

data = {'Name':name_list, "order":list(taxa.ORDER), "genus":list(taxa.GENUS),
        'stage':stage_names, 'stg':stages, "NOW_max_ma":list(taxa.MAX_AGE),  "NOW_mid_ma": list(age_list),  
        "NOW_min_ma":min_ma, "max_age":list(strat_stages.bottom[ind]),"mid_age":list(strat_stages.mid[ind]),
        'min_ma':list(strat_stages.top[ind]), "ID":list(taxa.LIDNUM), "col":list(taxa.NAME), 
        "NOW_stg":list(my_stage_numbers.astype(int)), "Mass":list(taxa.BODYMASS), "Diet_1":list(taxa.DIET_1), "Diet_2":list(taxa.DIET_2), "Diet_3":list(taxa.DIET_3)}

my_strat_data = {"stage_no":list(stage_list), "stage_bottom":list(my_stages_bottom), 
              "stage_mid":list(my_stages_mid), "stage_top":list(my_stages_top)}

NOW_strat = pd.DataFrame(my_strat_data)
NOW_strat.to_csv("NOW_strat.csv", index=False)
#print(np.max(taxa.MAX_AGE))
#print(min_ma)
occ_data = pd.DataFrame(data)


occ_data.to_csv("occ_strat.csv", index=False, quoting=2)

#plt.rcParams['figure.figsize'] = [36, 72]
#uncertainty = 0.01*taxa.MAX_AGE
#plt.hist(age_list)
#plt.plot(my_stage_numbers, taxa.mean_ma, yerr=uncertainty, xerr=None, fmt='none')
#ax = plt.gca()
#ax.tick_params(axis='x', labelrotation = 90)
#plt.show()


LIDNUM      False
NAME        False
MAX_AGE      True
MIN_AGE      True
ORDER        True
FAMILY       True
GENUS        True
SPECIES      True
BODYMASS     True
DIET_1       True
DIET_2       True
DIET_3       True
dtype: bool
LIDNUM      False
NAME        False
MAX_AGE     False
MIN_AGE     False
ORDER       False
FAMILY      False
GENUS       False
SPECIES     False
BODYMASS    False
DIET_1      False
DIET_2      False
DIET_3      False
dtype: bool
(32447, 12)
After cleanup:
(29758, 12)
